PYTHON-12. Работа с файлами разных форматов

# START

In [ ]:
import warnings; warnings.simplefilter('ignore')

import random
import collections

import pandas as pd
import numpy as np
import os
import re

import json  
from pprint import pprint
import xml.etree.ElementTree as ET

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


%matplotlib inline


In [ ]:
project_dir = r'/content/drive/My Drive/Colab Notebooks/data/u3/'
os.listdir(project_dir)

## 12.1 Формат JSON

In [ ]:
with open(project_dir + 'recipes.json') as f:
   recipes = json.load(f)

In [ ]:
pprint(recipes[0:3])

In [ ]:
len(recipes), type(recipes)

In [ ]:
recipes[0]['id']

In [ ]:
recipes[0]['ingredients']

In [ ]:
find_id = 13121

for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['cuisine'])
        break

In [ ]:
# Task 12.1.1

recipes[len(recipes) - 1]['cuisine']

In [ ]:
# Task 12.1.2
len(recipes[len(recipes) - 1]['ingredients'])


In [ ]:
# Task 12.1.3
find_id = 17636
for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['ingredients'])
        break



In [ ]:
# Task 12.1.4
find_id = 42013
for recipe in recipes:
    if recipe['id'] == find_id:
        print(len(recipe['ingredients']))
        break


In [ ]:
# Task 12.1.5
find_id = 23629
for recipe in recipes:
    if recipe['id'] == find_id:
        print(recipe['ingredients'])
        break

### Группировка данных

In [ ]:
cuisines = []  # создаём пустой список для хранения уникальных значений кухонь
for recipe in recipes:  # начинаем перебор всех рецептов
   if not(recipe['cuisine'] in cuisines):  # если тип кухни текущего блюда ещё не встречался
        cuisines.append(recipe['cuisine']) # добавляем его к списку cuisines
len(cuisines)


In [ ]:
# вариант с использованием множества

cuisines = set()  # создаём пустое множество для хранения уникальных значений кухонь
for recipe in recipes:  # начинаем перебор всех рецептов
   cuisines.add(recipe['cuisine']) # добавляем название типа кухни к множеству
len(cuisines)

In [ ]:
# Задание 12.3.1
# Сколько ингредиентов включено в состав всех блюд, описанных в наборе данных?

product_composition = set()

for recipe in recipes:
    for product in recipe['ingredients']:
        product_composition.add(product)

print(len(product_composition))

In [ ]:
# Задание 12.3.1
# Сколько ингредиентов встречается в рецептах блюд итальянской кухни ('cuisine' = italian)?

italian_product = set()

for recipe in recipes:
    if recipe['cuisine'] == 'italian':
        for product in recipe['ingredients']:
            italian_product.add(product)

print('italian product:', len(italian_product))

In [ ]:
# Задание 12.3.3
# Какие ингредиенты не встречаются в рецептах блюд русской кухни ('cuisine' = 'russian')?

russian_product = set()
check_list = set()
[check_list.add(x) for x in ['bacon', 'bread slices', 'buttermilk', 'red beets', 'mozzarella cheese', 'carrots']]

for recipe in recipes:
    if recipe['cuisine'] == 'russian':
        for product in recipe['ingredients']:
            russian_product.add(product)

check_list - russian_product

In [ ]:
food = collections.Counter()

for recipe in recipes:
    for item in recipe['ingredients']:
        food[item] += 1

food['sugar'], food['eggs']

In [ ]:
# Задание 12.3.4
# Какие ингредиенты входят в состав более чем 20% блюд, представленных в наборе данных?


top_products = set()
tresh_hold = 0.2 * len(recipes)

check_list = ['water', 'olive oil', 'salt', 'garlic', 'pepper']

for item, count in food.items():
    if count > tresh_hold:
        top_products.add(item)

for product in check_list:
    if product in top_products:
        print('[x]', product)
    else:
        print('[ ]', product)


In [ ]:
# Задание 12.3.5
one_recipe = set()
max_freq = 0
top_ingredients = ''

for item, count in food.items():
    if count > max_freq:
        top_ingredients = item
        max_freq = count
    elif count == 1:
        one_recipe.add(item)

top_ingredients

In [ ]:
# Задание 12.3.6
len(one_recipe)

## 12.4 Создание датафрейма

In [ ]:
df = pd.DataFrame(recipes)
df.sample(7)

In [ ]:
df.shape

In [ ]:
# Task 12.4.6

# import collections
# import pandas as pd

def find_item(cell):
    if item in cell:
        return 1
    return 0

# with open('recipes.json') as f:
with open(project_dir + 'recipes.json') as f:
   recipes = json.load(f)

df = pd.DataFrame(recipes)

food = collections.Counter()

for recipe in recipes:
    for item in recipe['ingredients']:
        food[item] += 1

ingredients = food.keys()


for item in ingredients:
    df[item] = df['ingredients'].apply(find_item)


df['ingredients'] = df['ingredients'].apply(lambda x: len(x))


In [ ]:
df.head()

In [ ]:
df.to_csv(project_dir + 'recipes.csv', index = False)

In [ ]:
# df.to_csv(project_dir + '1-recipes.csv', index = True)

## 12.5 Создание JSON-файла

In [ ]:
pprint(recipes[1:2])

In [ ]:
# Task 12.5.1
df = pd.read_csv(project_dir + 'recipes.csv')


In [ ]:
ids = []

for item in df.id.unique():
    ids.append(item)

type(ids), len(ids), ids[0:2]

In [ ]:
# Task 12.5.2

ingredients = []

for item in df.columns[3:]:
    ingredients.append(item)

# ingredients

In [ ]:
# Task 12.5.3
def make_list(incomin_row):
    ingredients = incomin_row.replace(0, np.nan).dropna(axis='columns')
    return list(ingredients.columns[3:])

ingredients = []

# отправим 1 строку датафрейма для проверки функции
ingredients = make_list(df.sample(1))

ingredients

In [ ]:
# Task 12.5.4

def make_list(one_df_row):
    ingredients = one_df_row.replace(0, np.nan).dropna(axis='columns')
    return list(ingredients.columns[3:])

new_recipes = []

ids = []

ingredients = []

for item in df.id.unique():
    ids.append(item)

# type(ids), len(ids)

for item in df.columns[3:]:
    ingredients.append(item)

# type(ingredients), len(ingredients)


for current_id in ids:
    cuisine = df[df['id'] == current_id]['cuisine'].iloc[0]
    current_ingredients = make_list(df[df['id'] == current_id])

    current_recipe = {'cuisine': cuisine, 'id': int(current_id), 'ingredients': current_ingredients}
    
    new_recipes.append(current_recipe)

pprint(new_recipes[0:3])



In [ ]:
with open(project_dir + "new_recipes.json", "w") as write_file:
    json.dump(new_recipes, write_file)

In [ ]:
os.listdir(project_dir)

## 12.7 Чтение Excel-файла

In [ ]:
data = pd.read_excel(project_dir + 'Fig3-1.xls', header=None)

In [ ]:
data.shape

In [ ]:
# with pd.ExcelFile('path_to_file.xls') as xls:  
#     data['Sheet1'] = pd.read_excel(xls, 'Sheet1', na_values=['NA'])  
#     data['Sheet2'] = pd.read_excel(xls, 'Sheet2')  

In [ ]:
# data = pd.read_excel('path_to_file.xls', ['Sheet1', 'Sheet2'])  

## 12.8 Продвинутое чтение Excel-файла

In [ ]:
data = pd.read_excel('http://www.econ.yale.edu/~shiller/data/Fig3-1.xls', header=None) 
# data


In [ ]:
data = pd.read_excel(project_dir + "nakladnaya.xls", header=None, skiprows=2)
data.shape

In [ ]:
data.dropna(how='all').shape

In [ ]:
data.dropna(how='all', inplace=True)
# data

In [ ]:
data.iloc[6:8, [1, 2, 6, 9, 11, 12]]

In [ ]:
data.iloc[6:8, [0, 2, 6, 9, 11, 12]]

In [ ]:
data.iloc[7:8, :].dropna(axis=1, how='all')

In [ ]:
data.iloc[6:8, :].dropna(axis=1, how='any')

In [ ]:
data.iloc[6:8, :].dropna(axis=1, how='all')

## 12.9 Запись в Excel-файл

In [ ]:
table.to_csv(project_dir + "table.csv") 

In [ ]:
table.to_excel(project_dir + "table.xls")  

In [ ]:
writer = pd.ExcelWriter(project_dir + 'test.xlsx')  
data.to_excel(writer, index=False, sheet_name='Таблица')  
writer.save()  

In [ ]:
os.listdir(project_dir)

In [ ]:
workbook = writer.book  
worksheet = writer.sheets['Таблица'] 

Важно запомнить, что при работе с Excel при задании диапазона значений значения указываются включительно. Именно поэтому мы выбрали во втором случае применение форматирования от столбца 1 до столбца 1.

In [ ]:

# workbook = writer.book  
# worksheet = writer.sheets['Таблица'] 

# money_fmt = workbook.add_format({'bold': True})  
# name_fmt = workbook.add_format({'color': 'red'})  
  
# worksheet.set_column('E:F', 20, money_fmt)  
# worksheet.set_column(1, 1, 20, name_fmt)  



## 12.10 Формат XML

In [ ]:
tree = ET.parse(project_dir + 'menu.xml')    
root = tree.getroot()  

In [ ]:
root

In [ ]:
root.getchildren()

In [ ]:
root.getchildren()[0] , root[0]

In [ ]:
root[0].attrib, root[1].attrib

In [ ]:
# узел price

root[0][0]

In [ ]:
root[0][0].text

In [ ]:
int(root[0][0].text), float(root[0][0].text)

In [ ]:

root[0][1].tag

In [ ]:
for elem in root:  
    for subelem in elem:  
        print(elem.attrib['name'], subelem.tag, subelem.text)  
    print() 

In [ ]:
len(root[1])

## 12.12 DataFrame и JSON из XML

In [ ]:
df_index = ['name', 'price', 'weight', 'class']  
df = pd.DataFrame(columns=df_index)  
  
for elem in root:  
    elements = [elem.get('name'), elem[0].text, elem[1].text, elem[2].text]  
    df = df.append(pd.Series(elements, index=df_index), ignore_index=True)

df

# END